Loads itemized individual contributions, calculates burn rates, and top fundraising days

In [1]:
import requests
import fecfile
import pandas as pd
import os
import datetime
from IPython.display import display

Time the notebook

In [2]:
start = datetime.datetime.now()

Set some viewing options

In [3]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 500)

Read dataframe of filings

In [4]:
filings = (
    pd.read_csv("../fetched/filings.csv")
)

filings.loc[ 
    lambda x: x['report_period'] == "Q3" ]

,committee_id,candidate_name,filing_id,cycle,form_type,date_filed,date_coverage_to,date_coverage_from,report_title,report_period,fec_uri,paper,amended,amended_uri,is_amendment,original_filing,original_uri,committee_type,contributions_total,cash_on_hand,disbursements_total,receipts_total
0,C00695510,Cory Booker,1358922,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00695510/1358922/,False,False,NaN,NaN,NaN,NaN,P,6017641.53,4223788.92,7159814.01,6023097.42
7,C00694455,Kamala Harris,1358880,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00694455/1358880/,False,False,NaN,NaN,NaN,NaN,P,11694156.88,10542691.73,14578958.89,11849290.45
13,C00703975,Joe Biden,1358905,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00703975/1358905/,False,False,NaN,NaN,NaN,NaN,P,15692077.08,8987627.81,17650871.16,15741432.01
17,C00696419,Amy Klobuchar,1358890,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00696419/1358890/,False,False,NaN,NaN,NaN,NaN,P,4805797.15,3679592.17,7836661.31,4806133.25
24,C00696948,Bernie Sanders,1357929,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00696948/1357929/,False,False,NaN,NaN,NaN,NaN,P,25252294.27,33734559.74,21559952.99,28025463.17
33,C00693234,Elizabeth Warren,1358497,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00693234/1358497/,False,False,NaN,NaN,NaN,NaN,P,24610428.73,25717673.76,18748452.02,24684963.30
41,C00697441,Pete Buttigieg,1358691,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00697441/1358691/,False,False,NaN,NaN,NaN,NaN,P,19144698.19,23378518.02,18501846.47,19211492.54
51,C00699090,Beto O'Rourke,1358876,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00699090/1358876/,False,False,NaN,NaN,NaN,NaN,P,4468814.60,3255729.87,6423123.17,4482284.35
59,C00693044,Julián Castro,1358889,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00693044/1358889/,False,False,NaN,NaN,NaN,NaN,P,3495405.71,672333.49,3960970.81,3497251.00
68,C00508416,John Delaney,1358358,2020,F3,2019-10-15,2019-09-30,2019-07-01,OCT QUARTERLY,Q3,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00508416/1358358/,False,False,NaN,NaN,NaN,NaN,P,464789.26,548060.86,7763004.07,868452.26


Get only the quarterlies

In [5]:
quarterlies = (
    filings
    # get quarters
    .loc[
        lambda x: x['report_period'].str.contains("Q", na=False)
    ]
    # remove filings that have been superceded by subsequent filings
    .loc[
        lambda x: x['amended'] == False
    ]
    .sort_values(['candidate_name', 'report_period'])
)

## Parse all the fec files with the filing list

### But first, more loading functions..

This function, when given a filing ID, returns only the contributions from individual donors:

In [6]:
def extract_contributions(filing_id):
    filing = fecfile.from_file(f"../fetched/filings/{filing_id}.fec")
    meta = filing['filing']
    
    # get only schedule A
    schedule_a = pd.DataFrame(filing["itemizations"]["Schedule A"])
    
    # remove time zone
    schedule_a["contribution_date"] = schedule_a["contribution_date"].dt.tz_localize(None)
    
    return (
        schedule_a
        
        # Extract only individual contributions
        .loc[lambda df: df["entity_type"] == "IND"]
        
        # Remove memo lines
        .loc[lambda df: df["memo_code"] == ""]
        
        .assign(
            filing_id = int(filing_id),
            
        )
        # filter schedule A
        [[
            "entity_type",
            "filer_committee_id_number",
            "filing_id",
            "transaction_id",
            "contribution_date",
            "contribution_amount",
            "contribution_aggregate",
            "contributor_organization_name",
            "contributor_first_name",
            "contributor_last_name",
            "contributor_zip_code",
            "contributor_state"
        ]]
    )

Create a unique ID out of first name, last name and 5-digit ZIP code

In [7]:
def make_donor_ids(df):
    return (
        df
        .assign(
            donor_id = lambda df: (
                df
                .assign(
                    zip5 = lambda df: (
                        df["contributor_zip_code"]
                        .fillna("-----")
                        .str.slice(0, 5)
                    )
                )
                [[
                    "contributor_first_name",
                    "contributor_last_name",
                    "zip5",
                ]]
                .apply(lambda x: (
                    x
                    .fillna("")
                    .astype(str)
                    # Remove periods, commas, extra whitespace
                    .str.replace(r"[\.,\s]+", " ")
                    .str.strip()
                    # Convert everything to upper-case
                    .str.upper()
                ))
                .apply("|".join, axis = 1)
            )
        )
    )

In [8]:
# takes concatenated contribution dataframe and merges with metadata
def get_metadata(contributions):
    candidate_filings = (
        filings
        [["filing_id",
        "candidate_name",
        "date_filed",
        "date_coverage_to",
        "date_coverage_from",
        "report_title",
        "report_period"]]
    )
    
    return (
        contributions
        .merge(
            candidate_filings,
            on = "filing_id",
            how = "left"
        )
    )

Concatenate all the filings data into one big DataFrame, and get individual contributions only

In [9]:
subset = quarterlies.loc[
    lambda x: x["candidate_name"] != "Wayne Messam" # He did not file a Schedule A
]

#subset = quarterlies

all_indiv = (
    pd
    .concat(
        [ extract_contributions(e) for e in subset["filing_id"] ]
    )
    .pipe( get_metadata )
    .pipe( make_donor_ids )
)

Remove the donors who have given $200 or less, or have been refunded to that level

In [10]:
# First build a frame of latest contributions
latest_contribs = (
    all_indiv
    .sort_values('contribution_date')
    .groupby([ 'candidate_name', 'donor_id'])
    .pipe(lambda grp: pd.DataFrame({
        "latest_contribution_aggregate": grp["contribution_aggregate"].last(),
        })
    )
    .reset_index()
)

latest_contribs.sort_values(
    'latest_contribution_aggregate',
    ascending = True
).head(3)


,candidate_name,donor_id,latest_contribution_aggregate
140991,Donald Trump,JOHN|WILLIAMS|95614,-4700.00
171314,Donald Trump,TOM|WALLACE|32034,-3974.79
174022,Donald Trump,WILLIAM|WEAVER|46228,-3100.00


Get list of individual contributions from individuals who have given more than $200, the official itemization threshold

In [11]:
threshold_indiv = (
    all_indiv
    .merge(
        latest_contribs,
        on = ['donor_id', 'candidate_name'],
        how= "left"
    )
    .loc[
        lambda x: x["latest_contribution_aggregate"] > 200
    ]
    .loc[
        lambda x: x["contribution_amount"] > 0
    ]
    .sort_values('latest_contribution_aggregate', ascending = False)

)

## Finally, some stats

What's the burn rate (`amount spent / amount raised`) for this period? (Not calculating donations from affiliated committees).

In [12]:
quarter = "Q3"

# Get latest quarterly
latest_period = (
    quarterlies
    .loc[ lambda df: df['report_period'] == quarter]
)

burn = pd.DataFrame({
    "candidate name": latest_period["candidate_name"],
    "cash on hand": latest_period["cash_on_hand"],
    "spent": latest_period["disbursements_total"],
    "raised": latest_period["contributions_total"],
    "difference": latest_period['contributions_total'] - latest_period['disbursements_total'],
    "percent spent": (( 
        latest_period['disbursements_total'] / latest_period['contributions_total'] 
    ) * 100).round(1)
}).sort_values('raised', ascending = False)


print(f"Candidate burn rates ({quarter} only)")
burn

Candidate burn rates (Q3 only)


,candidate name,cash on hand,spent,raised,difference,percent spent
129,Tom Steyer,2623142.12,47021989.49,49645130.22,2623140.73,94.7
24,Bernie Sanders,33734559.74,21559952.99,25252294.27,3692341.28,85.4
33,Elizabeth Warren,25717673.76,18748452.02,24610428.73,5861976.71,76.2
41,Pete Buttigieg,23378518.02,18501846.47,19144698.19,642851.72,96.6
13,Joe Biden,8987627.81,17650871.16,15692077.08,-1958794.08,112.5
118,Donald Trump,83216270.26,14479108.22,14272962.05,-206146.17,101.4
7,Kamala Harris,10542691.73,14578958.89,11694156.88,-2884802.01,124.7
75,Andrew Yang,6357361.37,4412923.43,9921124.76,5508201.33,44.5
0,Cory Booker,4223788.92,7159814.01,6017641.53,-1142172.48,119.0
17,Amy Klobuchar,3679592.17,7836661.31,4805797.15,-3030864.16,163.1


Biggest days for each candidate who has filed in Q3

In [13]:
# Get list of candidates
candidates = threshold_indiv.loc[ 
    lambda x: x["report_period"] == "Q3"
]['candidate_name'].unique()

for each in candidates:

    top_days = (
        threshold_indiv
        .loc[
            lambda x: x['candidate_name'] == each
        ]
        .groupby([
            pd.Grouper(
                key = "contribution_date",
                freq = 'D'
            ),
            "candidate_name",
        ])
        ["donor_id"]
        .nunique()
        .to_frame("donor_count")
        .reset_index()
        .sort_values('donor_count', ascending = False)
        [0:10]
        .assign(
            contribution_date = lambda df: df['contribution_date'].dt.strftime("%m/%d/%Y")
        )
    )

    print(f"Top 10 days for {each}")
    
    display(top_days)


Top 10 days for Joe Biden


,contribution_date,candidate_name,donor_count
5,04/25/2019,Joe Biden,3390
163,09/30/2019,Joe Biden,2070
158,09/25/2019,Joe Biden,1349
145,09/12/2019,Joe Biden,1241
162,09/29/2019,Joe Biden,1206
131,08/29/2019,Joe Biden,1186
102,07/31/2019,Joe Biden,1090
154,09/21/2019,Joe Biden,1059
103,08/01/2019,Joe Biden,1014
153,09/20/2019,Joe Biden,963


Top 10 days for Andrew Yang


,contribution_date,candidate_name,donor_count
261,09/30/2019,Andrew Yang,3331
260,09/29/2019,Andrew Yang,1849
243,09/12/2019,Andrew Yang,1431
244,09/13/2019,Andrew Yang,1363
256,09/25/2019,Andrew Yang,1270
258,09/27/2019,Andrew Yang,1045
259,09/28/2019,Andrew Yang,970
231,08/31/2019,Andrew Yang,950
255,09/24/2019,Andrew Yang,899
241,09/10/2019,Andrew Yang,876


Top 10 days for Pete Buttigieg


...

,contribution_date,candidate_name,donor_count
232,09/21/2019,Cory Booker,682
181,08/01/2019,Cory Booker,624
241,09/30/2019,Cory Booker,604
240,09/29/2019,Cory Booker,537
234,09/23/2019,Cory Booker,509
180,07/31/2019,Cory Booker,429
233,09/22/2019,Cory Booker,423
195,08/15/2019,Cory Booker,400
0,02/01/2019,Cory Booker,395
224,09/13/2019,Cory Booker,375


Top 10 days for John Delaney


,contribution_date,candidate_name,donor_count
189,07/31/2019,John Delaney,48
188,07/30/2019,John Delaney,31
59,03/14/2019,John Delaney,31
250,09/30/2019,John Delaney,28
201,08/12/2019,John Delaney,22
74,03/29/2019,John Delaney,20
220,08/31/2019,John Delaney,20
225,09/05/2019,John Delaney,20
210,08/21/2019,John Delaney,20
56,03/11/2019,John Delaney,19


Top 10 days for Joe Sestak


,contribution_date,candidate_name,donor_count
96,09/30/2019,Joe Sestak,40
20,07/14/2019,Joe Sestak,37
72,09/06/2019,Joe Sestak,24
7,06/30/2019,Joe Sestak,21
21,07/15/2019,Joe Sestak,19
0,06/23/2019,Joe Sestak,17
92,09/26/2019,Joe Sestak,15
13,07/07/2019,Joe Sestak,12
23,07/17/2019,Joe Sestak,11
26,07/21/2019,Joe Sestak,10


Check on timing

In [14]:
end = datetime.datetime.now()

d = (end - start)

f"The notebook ran for {round(d.total_seconds() / 60, 2) } minutes"

'The notebook ran for 10.37 minutes'

---

---

---